In [ ]:
!curl -L -o data/rice-image-dataset.zip https://www.kaggle.com/api/v1/datasets/download/muratkokludataset/rice-image-dataset

In [ ]:
!unzip -q data/rice-image-dataset.zip -d data/

In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Dispositivo disponible: {device}")

Dispositivo disponible: cpu


In [ ]:
!nvidia-smi

Mon Mar 18 12:49:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0              30W /  70W |    705MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## División de los conjuntos de train, val y test

In [ ]:
import splitfolders

input_dir = '/content/rice-dataset-sample'
output_dir = '/content/rice-dataset-sample-splits'

splitfolders.ratio(
    input_dir,
    output=output_dir,
    seed=42,
    ratio=(.7, .2, .1)
)

Copying files: 50 files [00:00, 1843.50 files/s]


## Modelos en timm

In [ ]:
import timm

In [ ]:
# Lista todos los modelos preentrenados de timm
timm.list_models('*', pretrained=True)[:20]

['bat_resnext26ts.ch_in1k',
 'beit_base_patch16_224.in22k_ft_in22k',
 'beit_base_patch16_224.in22k_ft_in22k_in1k',
 'beit_base_patch16_384.in22k_ft_in22k_in1k',
 'beit_large_patch16_224.in22k_ft_in22k',
 'beit_large_patch16_224.in22k_ft_in22k_in1k',
 'beit_large_patch16_384.in22k_ft_in22k_in1k',
 'beit_large_patch16_512.in22k_ft_in22k_in1k',
 'beitv2_base_patch16_224.in1k_ft_in1k',
 'beitv2_base_patch16_224.in1k_ft_in22k',
 'beitv2_base_patch16_224.in1k_ft_in22k_in1k',
 'beitv2_large_patch16_224.in1k_ft_in1k',
 'beitv2_large_patch16_224.in1k_ft_in22k',
 'beitv2_large_patch16_224.in1k_ft_in22k_in1k',
 'botnet26t_256.c1_in1k',
 'caformer_b36.sail_in1k',
 'caformer_b36.sail_in1k_384',
 'caformer_b36.sail_in22k',
 'caformer_b36.sail_in22k_ft_in1k',
 'caformer_b36.sail_in22k_ft_in1k_384']

In [ ]:
# Muestra el número de modelos preentrenados de timm
print(f"Número de modelos preentrenados en timm: {len(timm.list_models('*', pretrained=True))}")

Número de modelos preentrenados en timm: 1329


In [ ]:
# Muestra los modelos preentrenados de timm que empiecen por "resnet"
timm.list_models('resnet*', pretrained=True)[:10]

['resnet10t.c3_in1k',
 'resnet14t.c3_in1k',
 'resnet18.a1_in1k',
 'resnet18.a2_in1k',
 'resnet18.a3_in1k',
 'resnet18.fb_ssl_yfcc100m_ft_in1k',
 'resnet18.fb_swsl_ig1b_ft_in1k',
 'resnet18.gluon_in1k',
 'resnet18.tv_in1k',
 'resnet18d.ra2_in1k']

In [ ]:
# Muestra los modelos preentrenados de timm que empiecen por "vit"
timm.list_models('vit*', pretrained=True)[:20]

['vit_base_patch8_224.augreg2_in21k_ft_in1k',
 'vit_base_patch8_224.augreg_in21k',
 'vit_base_patch8_224.augreg_in21k_ft_in1k',
 'vit_base_patch8_224.dino',
 'vit_base_patch14_dinov2.lvd142m',
 'vit_base_patch14_reg4_dinov2.lvd142m',
 'vit_base_patch16_224.augreg2_in21k_ft_in1k',
 'vit_base_patch16_224.augreg_in1k',
 'vit_base_patch16_224.augreg_in21k',
 'vit_base_patch16_224.augreg_in21k_ft_in1k',
 'vit_base_patch16_224.dino',
 'vit_base_patch16_224.mae',
 'vit_base_patch16_224.orig_in21k',
 'vit_base_patch16_224.orig_in21k_ft_in1k',
 'vit_base_patch16_224.sam_in1k',
 'vit_base_patch16_224_miil.in21k',
 'vit_base_patch16_224_miil.in21k_ft_in1k',
 'vit_base_patch16_384.augreg_in1k',
 'vit_base_patch16_384.augreg_in21k_ft_in1k',
 'vit_base_patch16_384.orig_in21k_ft_in1k']

## Instanciar el modelo en timm

In [ ]:
model_name = "vit_base_patch16_224"
model = timm.create_model(model_name, pretrained=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
# model

In [ ]:
# Accede a la configuración de timm
model.default_cfg

{'url': '',
 'hf_hub_id': 'timm/vit_base_patch16_224.augreg2_in21k_ft_in1k',
 'architecture': 'vit_base_patch16_224',
 'tag': 'augreg2_in21k_ft_in1k',
 'custom_load': False,
 'input_size': (3, 224, 224),
 'fixed_input_size': True,
 'interpolation': 'bicubic',
 'crop_pct': 0.9,
 'crop_mode': 'center',
 'mean': (0.5, 0.5, 0.5),
 'std': (0.5, 0.5, 0.5),
 'num_classes': 1000,
 'pool_size': None,
 'first_conv': 'patch_embed.proj',
 'classifier': 'head'}

In [ ]:
for param in model.parameters():
  param.requires_grad=False

In [ ]:
import torch.nn as nn

in_features = model.head.in_features
out_features = 5

model.head = nn.Linear(in_features, out_features)
model.to(device)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [ ]:
params_to_update = []
for name, param in model.named_parameters():
  if param.requires_grad:
    params_to_update.append(param)
    print(name)

head.weight
head.bias


In [ ]:
# model

## Instanciar DataLoaders

In [ ]:
# Consigue el tamaño de imagen para entrenar el modelo
# TIP: Consulta la configuración del modelo
image_size = model.default_cfg["input_size"][1:3]

In [ ]:
from torchvision import transforms

transform_train = transforms.Compose([
    transforms.Resize(image_size),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_test = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
from torchvision import datasets

dataset_path = '/content/rice-dataset-sample-splits'

train_dataset = datasets.ImageFolder(root=f"{dataset_path}/train", transform=transform_train)
val_dataset = datasets.ImageFolder(root=f"{dataset_path}/val", transform=transform_test)
test_dataset = datasets.ImageFolder(root=f"{dataset_path}/test", transform=transform_test)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 16
shuffle = True

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)

## Entrenamiento de la red neuronal

In [ ]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(params_to_update)

In [ ]:
num_epochs = 15

best_val_loss = float("inf")
patience = 5
patience_counter = 0

for epoch in range(num_epochs):
  model.train()
  running_loss = 0.0
  for inputs, labels in train_dataloader:
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()

    outputs = model(inputs)

    loss = criterion(outputs, labels)
    loss.backward
    optimizer.step()

    running_loss += loss.item()

  epoch_loss = running_loss / len(train_dataloader)

  val_loss = 0.0
  model.eval()
  with torch.no_grad():
    for inputs, labels in val_dataloader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      val_loss += loss.item()

  val_loss /= len(val_dataloader)

  print(f"Epoch: {epoch+1} Train loss: {epoch_loss} Val loss: {val_loss}")

  # Early Stopping
  if val_loss < best_val_loss:
    best_val_loss = val_loss
    torch.save(model.state_dict(), "best_model.pth")
    patience_counter = 0
    print("Improved model")
  else:
    patience_counter += 1

  if patience_counter >= patience:
    print("Training stopped")
    break

Epoch: 1 Train loss: 1.647433916727702 Val loss: 1.859736442565918
Improved model
Epoch: 2 Train loss: 2.002789775530497 Val loss: 1.7712414264678955
Improved model
Epoch: 3 Train loss: 1.8413902918497722 Val loss: 1.795752763748169
Epoch: 4 Train loss: 1.5583130518595378 Val loss: 1.7437368631362915
Improved model
Epoch: 5 Train loss: 2.1013313134511313 Val loss: 1.7129195928573608
Improved model
Epoch: 6 Train loss: 1.8116962512334187 Val loss: 1.7110141515731812
Improved model
Epoch: 7 Train loss: 1.9720771312713623 Val loss: 1.6555240154266357
Improved model
Epoch: 8 Train loss: 1.6743003527323406 Val loss: 1.8528770208358765
Epoch: 9 Train loss: 1.8041248718897502 Val loss: 1.7675530910491943
Epoch: 10 Train loss: 2.113136649131775 Val loss: 1.9196304082870483
Epoch: 11 Train loss: 1.5824553767840068 Val loss: 1.8591095209121704
Epoch: 12 Train loss: 1.826436956723531 Val loss: 1.838700294494629
Training stopped


## Evaluar el modelo

In [ ]:
from sklearn.metrics import classification_report

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
  for inputs, labels in train_dataloader:
    inputs, labels = inputs.to(device), labels.to(device)

    outputs = model(inputs)

    _, preds = torch.max(outputs, 1)

    all_preds.extend(preds.numpy(force=True))
    all_labels.extend(labels.numpy(force=True))

print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

              precision    recall  f1-score   support

     Arborio       0.00      0.00      0.00         7
     Basmati       0.22      0.71      0.33         7
      Ipsala       0.00      0.00      0.00         7
     Jasmine       0.00      0.00      0.00         7
   Karacadag       0.60      0.86      0.71         7

    accuracy                           0.31        35
   macro avg       0.16      0.31      0.21        35
weighted avg       0.16      0.31      0.21        35



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
